In [1]:
notebook_id = "120"

<a id="ID_top"></a>
## Merge data baseline

This notebook merges data from Dynamic Gravity Dataset (gme 112), with the BRI membership dataset (113).

Work pending: add flow data from UNCOMTRADE

#### Notebook sections:
    
|| [0| Default imports](#ID_top) || [1|Part1](#ID_part1) || [2|Part2](#ID_part2) || [3|Part3](#ID_part3) || [4|Part4](#ID_part4) || [5|Part5](#ID_part5) ||

#### Import all packages that could be required

In [2]:
# %load s_package_import.py
# package library, use to ensure consistency across notebooks, refresh periodically
# general packages
import os # use with os.listdir(_path_)
import requests
import csv
import time
from datetime import datetime
from shutil import copyfile

#temp check
#from shutil import make_archive
#import zipfile

# data analysis packages
import pandas as pd
pd.options.display.max_columns = None # don't truncate columns
#pd.options.display.max_rows = None

import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import descartes

# custom scripts
import s_file_export
import s_filepaths
import s_un_comtrade_extract as s_un
import s_adj_matrix_plot

#=== network analysis
import networkx as nx
#=== gavity modelling
import gme as gme


/Users/antonios/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


#### Import module and declare path variables
`import s_filepaths.py`

In [3]:
# import ref file
import s_filepaths

# declare local variables to work with
path_raw = s_filepaths.path_raw
path_raw_dl = s_filepaths.path_raw_dl
path_store = s_filepaths.path_store
path_live = s_filepaths.path_live

<a id="ID_part1"></a>
### Part 1 | Load and merge datasets
|| [0| Default imports](#ID_top) || [1|Part1](#ID_part1) || [2|Part2](#ID_part2) || [3|Part3](#ID_part3) || [4|Part4](#ID_part4) || [5|Part5](#ID_part5) ||

In [4]:
# View all files in live directory
print(os.listdir(f"{path_live}"))

['121_input_di_matrix.csv.gzip', '.DS_Store', '112_input_gme_data.csv.gzip', '125_input_scope_country.csv.gzip', '120_input_scope_data.csv.gzip', '2_raw_explainer_doc.md', '121_input_de_matrix.csv.gzip', '113_input_bri_members.csv.gzip']


**GME dataset**

In [5]:
df_gme = pd.read_csv(f"{path_live}112_input_gme_data.csv.gzip",compression="gzip")
# preview gme data
df_gme.head()

year country_d iso3_d dynamic_code_d  landlocked_d  island_d   region_d  \
0  2005     Aruba    ABW            ABW             0         1  caribbean   
1  2006     Aruba    ABW            ABW             0         1  caribbean   
2  2007     Aruba    ABW            ABW             0         1  caribbean   
3  2008     Aruba    ABW            ABW             0         1  caribbean   
4  2009     Aruba    ABW            ABW             0         1  caribbean   

   gdp_pwt_const_d     pop_d  gdp_pwt_cur_d  capital_cur_d  capital_const_d  \
0        3906.5203  0.100031      4093.2434      23531.377        24173.982   
1        4118.1396  0.100830      4217.0669      25757.818        25396.307   
2        4196.4634  0.101218      4248.4707      27375.447        26631.465   
3        4433.6772  0.101342      4441.8828      28639.586        27871.596   
4        4183.0449  0.101416      4304.9224      29400.539        29122.635   

   gdp_wdi_cur_d  gdp_wdi_const_d  gdp_wdi_cap_cur_d  gdp_wdi_cap_const_d  \
0   2.331006e+09              NaN       23302.831988                  NaN   
1   2.421475e+09              NaN       24015.420612                  NaN   
2   2.623726e+09              NaN       25921.538234                  NaN   
3   2.791961e+09              NaN       27549.889422                  NaN   
4   2.498933e+09              NaN       24640.421244                  NaN   

       lat_d      lng_d  polity_d  polity_abs_d              country_o iso3_o  \
0  12.530384 -70.028992       NaN           NaN   Netherlands Antilles    ANT   
1  12.530384 -70.028992       NaN           NaN               Anguilla    AIA   
2  12.530384 -70.028992       NaN           NaN  Sao Tome and Principe    STP   
3  12.530384 -70.028992       NaN           NaN                Andorra    AND   
4  12.530384 -70.028992       NaN           NaN            Philippines    PHL   

  dynamic_code_o  landlocked_o  island_o         region_o  gdp_pwt_const_o  \
0          ANT.X             0         0        caribbean              NaN   
1            AIA             0         1        caribbean        348.76880   
2            STP             0         1           africa        391.01483   
3            AND             1         0           europe              NaN   
4            PHL             0         1  south_east_asia     458079.81000   

       pop_o  gdp_pwt_cur_o  capital_cur_o  capital_const_o  gdp_wdi_cur_o  \
0        NaN            NaN            NaN              NaN            NaN   
1   0.012903      365.93643   2.471682e+03     2.342796e+03            NaN   
2   0.160064      392.44177   1.101736e+03     3.205526e+03   1.458274e+08   
3        NaN            NaN            NaN              NaN   4.001201e+09   
4  91.641881   460142.72000   1.420047e+06     1.624159e+06   1.683346e+11   

   gdp_wdi_const_o  gdp_wdi_cap_cur_o  gdp_wdi_cap_const_o      lat_o  \
0              NaN                NaN                  NaN  12.250778   
1              NaN                NaN                  NaN  18.217348   
2     1.670446e+08         911.057012          1043.611485   0.989202   
3     3.675947e+09       46734.268282         42935.277871  42.500000   
4     1.854377e+11        1836.874120          2023.503659  11.817977   

        lng_o  polity_o  polity_abs_o  contiguity  agree_pta_goods  \
0  -69.301224       NaN           NaN           0                1   
1  -63.057232       NaN           NaN           0                1   
2    7.072665       NaN           NaN           0                0   
3    1.516486       NaN           NaN           0                0   
4  122.775020       8.0           8.0           0                0   

   agree_pta_services  agree_cu  agree_eia  agree_fta  agree_psa  agree_pta  \
0                   0         0          0          1          0          1   
1                   0         0          0          1          0          1   
2                   0         0          0          0          0          

In [6]:
# drop columns that we do not need
df_gme.columns

Index(['year', 'country_d', 'iso3_d', 'dynamic_code_d', 'landlocked_d',
       'island_d', 'region_d', 'gdp_pwt_const_d', 'pop_d', 'gdp_pwt_cur_d',
       'capital_cur_d', 'capital_const_d', 'gdp_wdi_cur_d', 'gdp_wdi_const_d',
       'gdp_wdi_cap_cur_d', 'gdp_wdi_cap_const_d', 'lat_d', 'lng_d',
       'polity_d', 'polity_abs_d', 'country_o', 'iso3_o', 'dynamic_code_o',
       'landlocked_o', 'island_o', 'region_o', 'gdp_pwt_const_o', 'pop_o',
       'gdp_pwt_cur_o', 'capital_cur_o', 'capital_const_o', 'gdp_wdi_cur_o',
       'gdp_wdi_const_o', 'gdp_wdi_cap_cur_o', 'gdp_wdi_cap_const_o', 'lat_o',
       'lng_o', 'polity_o', 'polity_abs_o', 'contiguity', 'agree_pta_goods',
       'agree_pta_services', 'agree_cu', 'agree_eia', 'agree_fta', 'agree_psa',
       'agree_pta', 'sanction_threat', 'sanction_threat_trade',
       'sanction_imposition', 'sanction_imposition_trade', 'member_eu_o',
       'member_wto_o', 'member_gatt_o', 'member_eu_d', 'member_wto_d',
       'member_gatt_d', 'member

Selecting columns that tell us basic descriptive information about the pair of countries. Ommitting membership of internationla organisations and mutual free-trade agreements

In [7]:
list_gme_columns_keep = ["year","country_d","iso3_d","dynamic_code_d",'landlocked_d',
                         'island_d', 'region_d', 'gdp_pwt_const_d', 'pop_d','lat_d', 'lng_d','country_o',
                         'iso3_o', 'dynamic_code_o','landlocked_o', 'island_o', 'region_o','pop_o','lat_o',
                         'lng_o', 'contiguity', 'distance']

In [8]:
df_gme_keep = df_gme.loc[:,list_gme_columns_keep].copy()
df_gme_keep.head()

year country_d iso3_d dynamic_code_d  landlocked_d  island_d   region_d  \
0  2005     Aruba    ABW            ABW             0         1  caribbean   
1  2006     Aruba    ABW            ABW             0         1  caribbean   
2  2007     Aruba    ABW            ABW             0         1  caribbean   
3  2008     Aruba    ABW            ABW             0         1  caribbean   
4  2009     Aruba    ABW            ABW             0         1  caribbean   

   gdp_pwt_const_d     pop_d      lat_d      lng_d              country_o  \
0        3906.5203  0.100031  12.530384 -70.028992   Netherlands Antilles   
1        4118.1396  0.100830  12.530384 -70.028992               Anguilla   
2        4196.4634  0.101218  12.530384 -70.028992  Sao Tome and Principe   
3        4433.6772  0.101342  12.530384 -70.028992                Andorra   
4        4183.0449  0.101416  12.530384 -70.028992            Philippines   

  iso3_o dynamic_code_o  landlocked_o  island_o         region_o      pop_o  \
0    ANT          ANT.X             0         0        caribbean        NaN   
1    AIA            AIA             0         1        caribbean   0.012903   
2    STP            STP             0         1           africa   0.160064   
3    AND            AND             1         0           europe        NaN   
4    PHL            PHL             0         1  south_east_asia  91.641881   

       lat_o       lng_o  contiguity     distance  
0  12.250778  -69.301224           0    120.05867  
1  18.217348  -63.057232           0    978.77728  
2   0.989202    7.072665           0   8563.69630  
3  42.500000    1.516486           0   7562.67330  
4  11.817977  122.775020           0  16904.59600

**BRI dataset**

In [9]:
df_bri = pd.read_csv(f"{path_live}113_input_bri_members.csv.gzip",compression="gzip")
# preview bri data
#df_bri.head()

In [10]:
# select columns to keep for merging
df_bri_keep = df_bri.loc[:,["iso_3","Year MOU sign","bri_flag"]]
# rename columns
df_bri_keep.columns = ["iso_3","bri_year_d","bri_flag_d"]
df_bri_keep.set_index("iso_3",inplace = True)
# preview
df_bri_keep.head()

bri_year_d  bri_flag_d
iso_3                        
AFG        2016.0           1
ALB        2017.0           1
DZA        2018.0           1
AGO        2018.0           1
ATG        2018.0           1

<a id="ID_part2"></a>
### Part 2 | Merge GME to BRI
|| [0| Default imports](#ID_top) || [1|Part1](#ID_part1) || [2|Part2](#ID_part2) || [3|Part3](#ID_part3) || [4|Part4](#ID_part4) || [5|Part5](#ID_part5) ||

In [11]:
df_gme_bri = df_gme_keep.merge(df_bri_keep,how = "left",left_on="iso3_d",right_on=df_bri_keep.index)

<a id="ID_part3"></a>
### Part 3 | Scope countries
|| [0| Default imports](#ID_top) || [1|Part1](#ID_part1) || [2|Part2](#ID_part2) || [3|Part3](#ID_part3) || [4|Part4](#ID_part4) || [5|Part5](#ID_part5) ||

In [12]:
df_scope = df_gme_bri.copy()
print(f"Basic stats | DF size: {len(df_scope)}")
df_scope.head()

Basic stats | DF size: 746048


year country_d iso3_d dynamic_code_d  landlocked_d  island_d   region_d  \
0  2005     Aruba    ABW            ABW             0         1  caribbean   
1  2006     Aruba    ABW            ABW             0         1  caribbean   
2  2007     Aruba    ABW            ABW             0         1  caribbean   
3  2008     Aruba    ABW            ABW             0         1  caribbean   
4  2009     Aruba    ABW            ABW             0         1  caribbean   

   gdp_pwt_const_d     pop_d      lat_d      lng_d              country_o  \
0        3906.5203  0.100031  12.530384 -70.028992   Netherlands Antilles   
1        4118.1396  0.100830  12.530384 -70.028992               Anguilla   
2        4196.4634  0.101218  12.530384 -70.028992  Sao Tome and Principe   
3        4433.6772  0.101342  12.530384 -70.028992                Andorra   
4        4183.0449  0.101416  12.530384 -70.028992            Philippines   

  iso3_o dynamic_code_o  landlocked_o  island_o         region_o      pop_o  \
0    ANT          ANT.X             0         0        caribbean        NaN   
1    AIA            AIA             0         1        caribbean   0.012903   
2    STP            STP             0         1           africa   0.160064   
3    AND            AND             1         0           europe        NaN   
4    PHL            PHL             0         1  south_east_asia  91.641881   

       lat_o       lng_o  contiguity     distance  bri_year_d  bri_flag_d  
0  12.250778  -69.301224           0    120.05867         NaN         NaN  
1  18.217348  -63.057232           0    978.77728         NaN         NaN  
2   0.989202    7.072665           0   8563.69630         NaN         NaN  
3  42.500000    1.516486           0   7562.67330         NaN         NaN  
4  11.817977  122.775020           0  16904.59600         NaN         NaN

In [13]:
# see all regions
df_gme_bri.region_d.unique()

array(['caribbean', 'middle_east', 'africa', 'europe', 'south_america',
       'pacific', 'southern_pole', 'south_asia', 'central_america',
       'south_east_asia', 'north_america', 'east_asia', 'eurasia',
       'central_asia'], dtype=object)

In [14]:
# remove islands
df_scope = df_scope[(df_scope.island_d == 0) & (df_scope.island_o == 0)].copy()

# focus on europe and asia
df_scope = df_scope[
    # filter all relevant destination countries
    ((df_scope.region_d.str.contains("europe")) | (df_scope.region_d.str.contains("asia")) | (df_scope.region_d.str.contains("middle"))) & 
    
    # filter all origin countries
    ((df_scope.region_o.str.contains("europe")) | (df_scope.region_o.str.contains("asia")) | (df_scope.region_o.str.contains("middle")))
                   ].copy()

# only include latest year (2016)
df_scope = df_scope[df_scope.year == 2016].copy()

# avoid same country pairs
df_scope = df_scope[df_scope.iso3_d != df_scope.iso3_o].copy()

In [15]:
print(f"Basic stats | DF size: {len(df_scope)}")
df_scope.head()

Basic stats | DF size: 7656


year             country_d iso3_d dynamic_code_d  landlocked_d  island_d  \
101  2016  United Arab Emirates    ARE            ARE             0         0   
209  2016            Azerbaijan    AZE            AZE             0         0   
233  2016               Belgium    BEL            BEL             0         0   
527  2016                 China    CHN            CHN             0         0   
701  2016        Czech Republic    CZE            CZE             1         0   

        region_d  gdp_pwt_const_d  pop_d      lat_d       lng_d country_o  \
101  middle_east              NaN    NaN  25.094378   55.454674  Portugal   
209       europe              NaN    NaN  40.362438   47.255833    Kuwait   
233       europe              NaN    NaN  50.691814    4.581812  Cambodia   
527    east_asia              NaN    NaN  35.389668  114.002470   Belarus   
701       europe              NaN    NaN  49.817062   15.696862  Holy See   

    iso3_o dynamic_code_o  landlocked_o  island_o         region_o  pop_o  \
101    PRT            PRT             0         0           europe    NaN   
209    KWT            KWT             0         0      middle_east    NaN   
233    KHM            KHM             0         0  south_east_asia    NaN   
527    BLR            BLR             1         0           europe    NaN   
701    VAT            VAT             1         0           europe    NaN   

         lat_o       lng_o  contiguity   distance  bri_year_d  bri_flag_d  
101  39.310741  -10.883486           0  6154.4214      2017.0         1.0  
209  29.279369   47.930035           0  1236.6086      2015.0         1.0  
233  12.320902  104.874400           0  9693.0625         NaN         NaN  
527  53.604687   27.802185           0  6904.3364      2013.0         1.0  
701  41.900013   12.447808           0   925.0072      2015.0         1.0

In [16]:
# compare destination and origin countries
print(f"Compare merged df country lengths: {len(df_scope.country_d.unique()) - len(df_scope.country_o.unique())}")
print(f"Compare merged df country members with set: {set(df_scope.country_d.unique()) - set(df_scope.country_o.unique())}")

Compare merged df country lengths: 0
Compare merged df country members with set: set()


<a id="ID_part4"></a>
### Part 4 | Export data in scope
|| [0| Default imports](#ID_top) || [1|Part1](#ID_part1) || [2|Part2](#ID_part2) || [3|Part3](#ID_part3) || [4|Part4](#ID_part4) || [5|Part5](#ID_part5) ||

In [17]:
# Save to live
filename = "scope_data"
s_file_export.f_df_export(df_scope,filename,p_file_id=f"{notebook_id}_")

Export | ../Data/1_raw_processed_backup/120_store_scope_data_20200809_1143.csv | COMPLETE
COPY   | ../Data/2_raw_processed_input/120_input_scope_data.csv.gzip | COMPLETE
